In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
#from matplotlib import animation
#from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [2]:
# read data

df = pd.read_csv('automatica_task_environments_all.csv', header=0,
                 skip_blank_lines=True, skiprows=[])

In [3]:
df

,ID,sequence_original,sequence_no_duplicates,sequence_clustered_silverware,sequence_clustered_drawers,coordinates_original,coordinates_clustered_silverware,coordinates_clustered_drawers,start_coordinates,strong_k,...,food_k,containment,error_original,length_original,error_no_duplicates,length_no_duplicates,error_clustered_silverware,length_clustered_silverware,error_clustered_drawers,length_clustered_drawers
0,a1,pocgkr,pocgkr,pocgz,pocgba,"p: (0.008034,0.957082,0.6890539999999999);o: (...","p: (0.008034,0.957082,0.6890539999999999);o: (...","p: (0.008034,0.957082,0.6890539999999999);o: (...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.723,6,0.723,6,0.666,5,0.723,5
1,a3,cgwpcffkkss,cgwpcfks,cgwpdz,cgwpdb,"c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.838,11,0.785,8,0.723,6,0.723,6
2,a5,kfsfksppwwggg,kfsfkspwg,zpwg,bpwg,"k: (-0.07699199999999999,0.733425,0.531662);f:...","k: (-0.07699199999999999,0.733425,0.531662);f:...","k: (-0.07699199999999999,0.733425,0.531662);f:...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.862,13,0.806,9,0.605,4,0.605,4
3,a11,ppfkswwkfsococggg,pfkswkfsococg,pzwzocndg,pbwbocndg,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.892,17,0.862,13,0.806,9,0.806,9
4,a13,wptgkfsoc,wptgkfsoc,wptgzoc,wptgboc,"w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.806,9,0.806,9,0.756,7,0.756,7
5,a16,pppwwwfffkkkggg,pwfkg,pwzg,pwbg,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.878,15,0.666,5,0.605,4,0.605,4
6,a19,ppoowwcceerr,powcer,powcz,powca,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.851,12,0.723,6,0.666,5,0.666,5
7,a22,cooppreerkk,coprerk,copz,copab,"c: (-0.7949970000000001,0.9232809999999999,0.4...","c: (-0.7949970000000001,0.9232809999999999,0.4...","c: (-0.7949970000000001,0.9232809999999999,0.4...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.838,11,0.756,7,0.605,4,0.666,4
8,a26,wwoppoffkkssskf,wopofkkskf,wopnz,wopnb,"w: (-0.196022,1.35572,0.466005);w: (-0.0570049...","w: (-0.196022,1.35572,0.466005);w1: (-0.057004...","w: (-0.196022,1.35572,0.466005);w1: (-0.057004...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.878,15,0.824,10,0.666,5,0.666,5
9,a29,opwpwgcoc,opwpwgcoc,opwqxgcnd,opwqxgcnd,"o: (-0.185,0.928,0.48103500000000005);p: (0.00...","o: (-0.185,0.928,0.48103500000000005);p: (0.00...","o: (-0.185,0.928,0.48103500000000005);p: (0.00...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.806,9,0.806,9,0.806,9,0.806,9


### Define functions for prediction

In [4]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [5]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        #print(objects)
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [6]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10,
                    seq='sequence_original', coords='coordinates_original', error='error_original'):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,seq])
        strong_k = list(data.at[row,'strong_k'].split(','))
        mid_k = list(data.at[row,'mid_k'].split(','))
        #food_k = list(data.at[row,'food_k'].split(','))
        food_k = []
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,coords].split(';'))}
        #print(coordinates)
        start_coordinates = list(ast.literal_eval(data.at[row,'start_coordinates']))
        sequence = str(data.at[row,seq])
        c1 = {obj: 1.2 for obj in objects}
        k1 = {obj: 1.0 for obj in objects}
        
        #for k2 in np.arange(1.1,2.0,0.1):
            #k_food = round(k2,2)
            #k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                #for c in np.arange(1.0,2.0,0.1):
                #    c = round(c, 1)
                #    c1 = {obj: 1.2 for obj in objects}
                
            for dim in dimensions:                
                # get average edit distance
                median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                median = median / len(sequence)
                #params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                params = 'c: 1.2' + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,error]
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df for original sequences

In [7]:
# ~3 min for n=100
%time results_original = get_avg_editdist(df,n=100)

CPU times: user 3min 24s, sys: 862 µs, total: 3min 24s
Wall time: 3min 24s


In [28]:
results_original.T.to_csv('results/results_original_all_n100_2020-11-03.csv', header=True, index=True)

### Calculate edit distances, create df for sequences w/o duplicates

In [9]:
# ~3 min for n=100
results_noduplicates = get_avg_editdist(df,n=100, seq='sequence_no_duplicates',
                                        error='error_no_duplicates')

In [30]:
results_noduplicates.T.to_csv('results/results_noduplicates_all_n100_2020-11-03.csv', header=True,index=True)

### Calculate edit distances, create df for clustered sequences (silverware)

In [11]:
results_silverware = get_avg_editdist(df,n=100, seq='sequence_clustered_silverware',
                                    coords='coordinates_clustered_silverware', 
                                    error='error_clustered_silverware')

In [29]:
results_silverware.T.to_csv('results/results_silverware_all_n100_2020-11-03.csv', header=True, index=True)

### Calculate edit distances, create df for clustered sequences (drawers)

In [13]:
results_drawers = get_avg_editdist(df, n = 100, seq = 'sequence_clustered_drawers',
                                    coords = 'coordinates_clustered_drawers',
                                    error = 'error_clustered_drawers')

In [31]:
results_drawers.T.to_csv('results/results_drawers_all_n100_2020-03-11.csv',header=True,index=True)

### Get lowest error, compare edit distances for all variants

In [15]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [37]:
# original sequences

lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_original)
lowest, lowest_idx, np.mean(results_original['error'])

(0.7108442291242113,
 Index(['c: 1.2; k: 0.7,0.8; z', 'c: 1.2; k: 0.8,0.9; z'], dtype='object'),
 0.8318749999999998)

In [34]:
# original sequences w/o duplicates

lowest_nodup, lowest_idx_nodup, list_mean_nodup, results_mean_nodup = get_lowest_error(results_noduplicates)
lowest_nodup, lowest_idx_nodup, np.mean(results_noduplicates['error'])

(0.6632115943696825,
 Index(['c: 1.2; k: 0.2,0.3; xy', 'c: 1.2; k: 0.2,0.3; xyz',
        'c: 1.2; k: 0.3,0.4; xy', 'c: 1.2; k: 0.3,0.4; xyz'],
       dtype='object'),
 0.7759249999999994)

In [36]:
# silverware clustered

lowest_s, lowest_idx_s, list_mean_s, results_mean_silverware = get_lowest_error(results_silverware)
lowest_s, lowest_idx_s, np.mean(results_silverware['error'])

(0.6163358516483515,
 Index(['c: 1.2; k: 0.8,0.9; xy'], dtype='object'),
 0.6876374999999993)

In [19]:
# drawers clustered

lowest_d, lowest_idx_d, list_mean_d, results_mean_drawers = get_lowest_error(results_drawers)
lowest_d, lowest_idx_d, np.mean(results_drawers['error'])

results_mean_drawers.T.to_csv('results/results_mean_noduplicates_all_n100_2020-11-03.csv',header=True,index=True)

(0.6504718545343542,
 Index(['c: 1.2; k: 0.7,0.8; xz', 'c: 1.2; k: 0.8,0.9; xz'], dtype='object'),
 0.7144303797468347)

### Plot best model for all variants

In [23]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error_original = df['error_original']
error_noduplicates = df['error_no_duplicates']
error_silverware = df['error_clustered_silverware']
error_drawers = df['error_clustered_drawers']

seqs_original = results_mean['sequence'][:-1].values
seqs_noduplicates = results_mean_nodup['sequence'][:-1].values
seqs_silverware = results_mean_silverware['sequence'][:-1].values
seqs_drawers = results_mean_drawers['sequence'][:-1].values

res_original = results_original['c: 1.2; k: 0.7,0.8; z'][:-1].values
res_noduplicates = results_noduplicates['c: 1.2; k: 0.2,0.3; xy'][:-1].values
res_silverware = results_silverware['c: 1.2; k: 0.8,0.9; xy'][:-1].values
res_drawers = results_drawers['c: 1.2; k: 0.7,0.8; xz'][:-1].values


x_original = [x for x in range (0,len(seqs_original))]
x_noduplicates = [x for x in range (0,len(seqs_noduplicates))]
x_silverware = [x for x in range (0,len(seqs_silverware))]
x_drawers = [x for x in range (0,len(seqs_drawers))]

# create subplots
fig, axs = plt.subplots(nrows=4, ncols=1, sharex=True)

# plot scatter + lines for all simulations
axs[0].scatter(x_original, res_original, marker='o', s=20, c='blue', alpha=0.5, label='model-generated (avg. 0.71)')
#axs[0].plot(x_original, res_original, c='blue', alpha=0.5)
axs[1].scatter(x_noduplicates, res_noduplicates, marker='o', s=20, c='darkgoldenrod', alpha=0.5, label='model-generated (avg. 0.66)')
#axs[1].plot(x_noduplicates, res_noduplicates, c='darkgoldenrod', alpha=0.5)
axs[2].scatter(x_silverware, res_silverware, marker='o', s=20, c='green', alpha=0.5, label='model-generated (avg. 0.62)')
#axs[2].plot(x_silverware, res_silverware, c='green', alpha=0.5)
axs[3].scatter(x_drawers, res_drawers, marker='o', s=20, c='darkmagenta', alpha=0.5, label='model-generated (avg. 0.65)')
#axs[3].plot(x_drawers, res_drawers, c='darkmagenta', alpha=0.5)

# error function + connection for original
axs[0].scatter(x_original, error_original, c='red', s=20, marker='o', alpha=0.5, label='baseline (avg. 0.83)')
axs[0].plot(x_original, error_original, c='red', alpha=0.5)
axs[0].plot((x_original,x_original),(error_original,res_original), '--', c='black', alpha=0.6)

# error function + connection for no duplicates
axs[1].scatter(x_noduplicates, error_noduplicates, c='red', s=20, marker='o', alpha=0.5, label='baseline (avg. 0.78)')
axs[1].plot(x_noduplicates, error_noduplicates, c='red', alpha=0.5)
axs[1].plot((x_noduplicates,x_noduplicates),(error_noduplicates,res_noduplicates), '--', c='black', alpha=0.6)

# error function + connection for silverware
axs[2].scatter(x_silverware, error_silverware, c='red', s=20, marker='o', alpha=0.5, label='baseline (avg. 0.69)')
axs[2].plot(x_silverware, error_silverware, c='red', alpha=0.5)
axs[2].plot((x_silverware,x_silverware),(error_silverware,res_silverware), '--', c='black', alpha=0.6)

# error function + connection for drawers
axs[3].scatter(x_drawers, error_drawers, c='red', s=20, marker='o', alpha=0.5, label='baseline (avg. 0.71)')
axs[3].plot(x_drawers, error_drawers, c='red', alpha=0.5)
axs[3].plot((x_drawers,x_drawers),(error_drawers,res_drawers), '--', c='black', alpha=0.6)

plt.suptitle('Comparison of simulation variants', fontsize=14, y=0.92)
axs[3].set_xticks(x_original)
axs[3].set_xticklabels(IDs, rotation=90, fontsize=6)

axs[0].set_ylabel('original', fontsize=10)
axs[1].set_ylabel('no duplicates', fontsize=10)
axs[2].set_ylabel('silverware', fontsize=10)
axs[3].set_ylabel('drawers', fontsize=10)
#axs[3].set_xlabel('sequence', fontsize=14)

axs[0].legend(fontsize=8)
axs[1].legend(fontsize=8)
axs[2].legend(fontsize=8)
axs[3].legend(fontsize=8)

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

In [21]:
diff_original = error_original - res_original
diff_silverware = error_silverware - res_silverware
diff_drawers = error_drawers - res_drawers
diff_noduplicates = error_noduplicates - res_noduplicates

In [22]:
stat, p = wilcoxon(diff_original, diff_drawers, zero_method='wilcox')
print('Original vs drawers: W = %.3f, p = %.5f' % (stat, p))

stat, p = wilcoxon(diff_original, diff_silverware, zero_method='wilcox')
print('Original vs silverware: W = %.3f, p = %.5f' % (stat, p))

stat, p = wilcoxon(diff_original, diff_noduplicates, zero_method='wilcox')
print('Original vs no duplicates: W = %.3f, p = %.5f' % (stat, p))

stat, p = wilcoxon(diff_noduplicates, diff_silverware, zero_method='wilcox')
print('No duplicates vs silverware: W = %.3f, p = %.5f' % (stat, p))

stat, p = wilcoxon(diff_noduplicates, diff_drawers, zero_method='wilcox')
print('No duplicates vs drawers: W = %.3f, p = %.5f' % (stat, p))

Original vs drawers: W = 886.000, p = 0.00111
Original vs silverware: W = 1237.000, p = 0.06621
Original vs no duplicates: W = 1440.500, p = 0.75677
No duplicates vs silverware: W = 1008.000, p = 0.04091
No duplicates vs drawers: W = 1014.000, p = 0.01331
